In [1]:
import pandas as pd
import re
from collections import Counter
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import pytz

In [2]:
# Read the list of filenames from the configuration file
with open('../file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'

# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = f"../data/{file}"
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])


In [3]:

data['date'] = pd.to_datetime(data['date'])


In [4]:

def convert_utc_to_cet(df, date_column='date'):
    # Make a copy to avoid modifying the original
    df = df.copy()
    
    # Ensure timestamps are UTC aware
    if df[date_column].dt.tz is None:
        df[date_column] = df[date_column].dt.tz_localize('UTC')
    elif df[date_column].dt.tz != pytz.UTC:
        df[date_column] = df[date_column].dt.tz_convert('UTC')
    
    # Convert to CET/CEST (Europe/Berlin includes proper DST handling)
    df[date_column] = df[date_column].dt.tz_convert('Europe/Berlin')
    
    return df



In [5]:
data = convert_utc_to_cet(data)

In [6]:
data["user"] = data["user"].replace("Banties1g", "banties_x")
data["user"] = data["user"].replace("banties1g", "banties_x")
data["user"] = data["user"].replace("chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("CHILI_POE", "chili_con_bacon")
data["user"] = data["user"].replace("Chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("chili_conbacon", "chili_con_bacon")
data["user"] = data["user"].replace("Wirelesss_", "W1r3lesss")
data["user"] = data["user"].replace("treklul", "trek44_")
data["user"] = data["user"].replace("ttrek_", "trek44_")
data["user"] = data["user"].replace("trek_x", "trek44_")
data["user"] = data["user"].replace("TriplesingleJ", "TripleSingleJames")
data["user"] = data["user"].replace("uwu_cougar", "uuccugr")
data["user"] = data["user"].replace("uuccugr_","uuccugr")
data["user"] = data["user"].replace("StanIV4_", "stan_iv4")
data["user"] = data["user"].replace("Muuskie2", "Muuskie")
data["user"] = data["user"].replace("nishad_more1311", "nishad13")
data["user"] = data["user"].replace("softarballt", "softarr")
data["user"] = data["user"].replace("softarballtt23", "softarr")
data["user"] = data["user"].replace("lajosbarnabas", "lajoss__")
data["user"] = data["user"].replace("Bonkwiththefunk", "bonk67")
data["user"] = data["user"].replace("qfishyy11", "bonk67")

In [7]:

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}

# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [8]:
# 7tv list
tv7_emotes = """omeFaded nonono wideSpeedFear wideSpeedDesert SpeedOmg Disappointed ME? ome13 doroPfft CONFUSED doro52 wideSpeedConcerned doroBANGER doroVibe DoroBrainPLug NAHHHH intrigued oda WOOF BANG peepoFirework Gladge omeMoji SobBounce belka doroHips dorochan doroXi DOMEYES DOWAY FRENCH YeahIGetIt doroPls Offline sheMightBeRight 89 dwerk wideGkeyDance3 76 xqcCheer omeCheer Dmile zoro LMAO iguessbro doroScary haii SON omeBru wideSpeedCall plink laracroft ADHD yipe doroWaiting 67 FOWL doroLOL OfCourse tyler67 THYME peepoLost speed61 GOODMOOD Jercula ILOVECS CLOWN thatsCrazy xqcBleh omEING wideSpeedLaugh15 WHATTHEHELI RickyDicky jonkler wideSpeedPumpkin wideSpeedLaugh16 SPIDER chillCat LEEKED soblaugh BANANA wideSpeedLaugh21 Binoculars footstepmenace ome99 doroBanger GhankYou NO YES auntiePls OMEGADANCEBUTGHAST gkeyRide GENERATINGGODSEED perimeter sideeye omePfft MySunshine posture ??? twocatsfightingonacouch dSmile wideSpeedLaugh4 flirtt Suspicion zigzag YeatCat nom omeSad shomonting thinking agafat FirstTimeClanka DORVIS gasp RISING dd uncPLS domgBruh Deafge OMEGADANCEBUTFASTER kim3 Buggin speed8 .... d32 ome54 ohok minionBike Clown Explosion hackingCD JermaSoy MathTime MoneyRain PokiShare TakingNotes :0 :3 :33 :tf: !boost +1 0pixel 1DLove 3Head 3Heading 4House 4Shrug 4Weird 5Head AAAA Acknowledged ACTINUP ADHD agahi AIM AIRBALL AIWITHTHEBRAIDS Alarm ALE Alfred AlienDance AlienPls AlienPls2 AlienPls3 Aloo Alright amongE ANGRE ANOTHERONE Ant AREYOUAGIRL AREYOUAGIRLFtxQcYellingAtYou areyoufr AreYouSeriousRightNeow arnoldHalt arthur Assept AURA AwHellNah Aware AWOO AWOOGA axelF ayo bah BAND Banger BANGER banties Barack barryArrive Barry63 BantiesPaulBeef Based BASED BatChestAbove batman Batman batJAM batPls Beatles Bedge BEG BEGGING Bello BigD bieberDougie BELIEVERS BibleThump BINGO Bleh Bloons BOOBA bog BOOM BOOMIES BOINK BORGIR Borfday brbToilet Broadcaster brb Bruh BRUHMM bruv buh buhbye buhFlipExplode BUSSIN BUSSERS bye CanIHaveADollar cannySilly catAsk catBusiness CatCozy catDespair catEat catJAM catKiss catPls catSigh catSmash CatTime catTwerk CAUGHT Caught CaughtIn4K Celebrating CHADDING characterSelected CHATTERS chatting cheerleaders ChillGuy Chillin Cinema chilling clappi Clap classic CLEAN Clueless CLIPPERS CLOWNDETECTED COCKA cokeBreak COMEHERE Concerned Considering Cooked COPIUM crabPls Crunch CS2 Cuck Cuh D: damily Damn dansi dash Dave deadassFaint Delusional DemonTiming Dentge despair DespairRyan Devious DIESOFCRINGE Dime DinkDonk Dinema doggoSlava dogJAM DogLookingWickedAndCool doid dojaPls dome44 dome32 donowall donoWall doroAunt doroBleh dorobubu doroCD DoroCheer doroFiddy doroFlex doroGHOST doroHEAD doroKick doroL doroMAD doroPray doroRip doroSoy DoroTalkingAgain Dorozea doster DOUBTERS DRAIN Drake DRAMA dreamwastaken drooling drukiDnace drukiDnace2 duaKiss dudWhat EDGE EDM EDITING emo erinNya essaying ewphop eww EZ EZdodge Exerpas Explosion eyeroll fadedthanaho FARMING FeelsBadMan FeelsDankMan FeelsBlackScreen FeelsGladMan FeelsLagMan FeelsLateMan FeelsOkayMan FeelsStrongMan FeelsTiredMan FeelsWeirdMan FeelsWowMan fein FEINFEINFEINFEINFEINFEINFEINFEI FellOff fembajJAM Fiddy FiddyWtf FINALLY firewriting FirstTime FirstTimeBackseating FirstTimeChadder FirstTimeChatter FirstTimeEmoteFail FirstTimeGooner FirstTimePepega FirstTimeTest firsttimebuh FLASHBANG flightnotL Flirt Flushed fnaf footstep forsenCD forsenLaughingAtYou ForsenSingingAtYou forsenPls fortnite fr freakbob freakyfredday freddy Freedom FUNNY g32 GAGAGA gachiGASM gachiHYPER gamily GAMBA GameplayTime GAMING GatieG Gaught GENIUS GetALoadOfThisGuy gg GIGACHAD GIGACHAIR GIGACLAUS GIGAMODS GIGAMOD gigl gkeyFlip gkeyPregnantBounce gkeySMP gkeyUwu gkeywide gkeyWiding gkitten GivenUp girlBoss gkitten glorpaga glorpdetective glorp GlorpMeeting glorprave gmoney goaler goat goblin44 GODDID Gogging GoodBye Gooner gooner GoodTake GOONING GotCaughtTrolling GotEEM gothKiss gPls greetingsladies GREEDY GROOTING GRRR GULP GuitarTime GYAT HABIBI hackingCD HACKERMANS hai HAH HaltEinfachDeineFresseDuHurensohn HandsUp happi HARAM HarryStylesKiss Headbang healed HECOOKING heh HEHE HEHEHEHA Heisenberj HELLO HELP helvete Herewego hesRight heyywithrizz HEYYY hi hiii hiiii Hmm HOBBY HOLY HolyFuck homelessPOV HowDoWeTellHer HowDoWeTellHim hue HUH HUHHHHHHHHHH iAsked ICANT idiot iDrive IFISPEAK IfYouCantSeeThisEmoteUseExclamationMark7tv Ignored IGON imback IMAGINENOTHAVING7TVGETFUCKEDNON7TVUSERSIMAGINENOTHAVING7TVGETFUCKEDNON7TVUSERSIMAGINENOTHAVING7TVGET ImNotOk ImtiredBoss INTENSEGAMING islandboy ISeeYou itsover itstime Jackass jacob1 jacob2 jacob3 jacob4 jah Jammies JARVIS Jay JermaSoy jiggy job JOB Joel joever john Johnporkiscalling JokerHAHA JokerLaugh juh JumpScared JUMPSCARE JustAChillGuy JustAnotherDay JustHowItIs justinbieber KaiCenatOhiogyatwithskibiditoiletwatchingtheWrizzhappeningrightinfrontofhimwithfanumtaxtaxingthegyat KanyeStare KEKW KENOUGH KeyShaker kim3 kittyBANGER kittyBop KKalinka KKonaW KKool kratos Lamonting LastTimeChatter lava lebronArrive lebronJAM lebronTROLL LEBRONNN lemon Lemon LetsBingo LETHERCOOK LETSFUCKINGJOE LETSGO LieMeter life Life Listening LiterallyMe Lithuanian LittleTrolling LiveReaction LL LMAOFREAKY lmao Loading LOCKIN lockedin LOL Looking LookUp lore luh lurkk luton LULE LULW MAJ Madge ManchesterUnited Massive? MarblesTime Martin matSad maxwin MeRN me: MeWhenIBuyEgyptianProperty MEGALUL mee Memories merch mhm MicTime mikuPls mindloud modCheck ModAbuse MODDING Modding mods MODS Mog monakS monday monkeyListening monkeySip MONKA monkaTOS monkaW MONKE MinionHoting MoneyRain muted mutted MUGA MVP MVPOfFarallah MYHEARTILOVEDHER myIQ MYLIFE NAILS NAILSING NAHH NAHHH NAHHHH nananAYAYA NAUR NAvsEU Nerd niceguy NOCHECKMARKS NODDERS NOIDONTTHINKSO NoMaidens NOOPERS NOOOOO NOHORNY noonecares NOSHOT NOTED notListening notxqcL NOW NOWAY NOWAYING np nt nuhuh nyehehehe nyanPls nya o7 Ogre ohhh ohhhhhhhhh OHMYGAWDD ohneFinger ohno ohSHIT oj OK Okei okak OLDWORK OM om omE ome10 ome101 ome104 ome105 ome14 ome15 ome18 ome21 ome29 ome32 ome4 ome41 ome44 ome44444444 ome47 ome5 ome51 ome52 ome55 ome57 ome67 ome69 ome79 ome808 ome83 ome9 ome90 ome96 ome99 OMEGADANCE OMEGALUL OMEGALULiguess omEE omeJAM omeJudging omeOhSHIT omeScrajj omeStare OMEYES omeWiggle OMFG omgBruh ongang OneGuy ONEMORE OnMyWayToDoroMomHouse OOOO oopsie otag OuttaPocket OVERWATCH OVERWORKING OverwhelminglyWholesome owoCheer PagBounce PagChomp PagMan Panam parasocial Parasocial PARASOCIAL paris paul Paul paulNya PauseMan PAUSENEMOGU Peace PEEPEES peepoAds peepoBox peepoBelievers peepoClap peepoComfy peepoDJ peepoDoubters peepoEvil peepoFarmer peepoFat peepoGiggles peepoHappy peepoHey peepoHug peepoKiss peepoLeave peepoLegs peepoLove peepoMarch peepoPls peepoPride peepoRiot peepoSad peepoShy peepoSmile peepoStop peepoTalk pepeAgony pepeGun PepeHands pepeJAM PepeLaugh pepePoint PepePls pepeW Pepega PepegaAim PepegaChat PepegaReading PepoG Petter Pffttt Pffttt2 phew phpk pickle PianoTime Pipege pKitten pL Please pleading plink-182 plinkVibe plonk pmo Plotge PogO PogU pointless pokiFlirt pol POLICE Pondering popipopipipopipo poroPls POVbornbefore2000 ppHop ppL ppOverheat Prayge prePffttt PRIMERS PTSD pulNya PuzzleTime qq ragebait RAGEY RAHH RainTime RAMBOLMG RareParrot ratomilton RaveDance RaveTime ratio Reacting RealForsen ReallyMad RebeccaBlack Reddit RememberTheDays RibertJam RiddleMeThis RIPBOZO RIRI Rizzler RobertJam ROFL RoxyPotato RUNNING rt ryanArrive Sadding Sadge SADge SAJ SAVEME SCATTER saythatagain scawy SCHEISSE SCHIZO SCRAPETHATSHITJOHNNY SCHTOP sdd SERIOUSLY SEXO shogaNya Shits shutup Shruge silliness sisyphus Sippin Sits skip SLAY Sleepo Smile smh Smoge SmurfHey Smurfing SNACKING SNEAK SNIFFA sob SOLARFLARE songbird sotruebestie SOYSCREAM Speechless speed1 speed2 speed21 speed25 speed32 speed4 speed44 speed8 speed88 speedVibe spfLEAN:()wiltee_()tonyhawkproskater4:-:-:_FREEWAVE3-encinoman--:enteringwalmart:-wheezethelean-123 SpeedLaugh SpeedLeft SpeedR spongePls squadHips Stare Staring steve Steve SteerR StreamEnding STREAMER STREAMERSGIVINGTHEWORSTFUCKINGTAKESINEXISTENCE StoryTime Surfing SurE sus susDog Susge SUSSY Swag swagJAM ta tak TakingNotes TeamEDWARD test THATHIT ThatsJustMe ThePaulers TheVoices TheWolfInMe Thinking Thinking2 ThisChat ThisIsMinecraft TIMEOUT Tomfoolery totallylistening TriJam TriKool TriSad TRUEING TRIVSsorry ts Tuckge tuff TWEAK typeshit typhu UGH um UltraMad unibrow unemployment unmod uwu uuh VALORANT veryDoro VeryKey VeryPog VeryPogftxQcInTheShower vibePls VibePls VIDEOGAME VIEWERS vips Voices wade Waddup waga wah waiting Waiting WAHHH WAJAJA WAIT WAITWAITWAIT WakeTheFuckUpSamuraiWeHaveACityToBurn wallE waltuh walterShocked WalterVibe War WasZumPenis WATAFUCKEDUPDAY WatchingStream WAYTOODANK wdym WeAreLive WeDoNotCare WEDIDIT WEEWOO WeGood WePaid WHATAFUCKEDUPDAY WHAT WHATTT wheresmyhug Whenyourinnerwolfreleases WideAlERT WideCatGroove wideDvaAss WideHardo WidelebronJAM widemonkaGIGAftRobertDowneyJr wideprespeedlaugh WideRaveTime wideReacting wideSpeedLaugh3 widetime WidezyzzPls wig WineTime winton Wisdom woah Wokege WOT wot wrapitup WW wowie Xd xar2EDM xdd XDoubt xJAM xqc32 xqcBOZO xqcDespair xqcFuel xqcGoofy xqcL xqcSCHIZO xqcSlam xqcTake xqcTwerk xQcVeryWide YAAAY YamesBond YANITED YAPPING YeahThatsWhatIWouldaDid YEAHHH YEP YESS YIPIEE yonose Yoink YOOLOOKATTHISCATDOINITSLILDANCYDANCEINTOABREAKDANCEMOVE Yooo YOUDIED YouGotMe YouWouldntGetIt ZAMN ZhongXina zyzzBass zyzzJAM"""

# Convert to list and clean 
emote_list = [emote.strip() for emote in tv7_emotes.split() if emote.strip()]

# Remove any remaining duplicates (if any)
unique_emotes = sorted(list(set(emote_list)))

print(f"Total unique emotes: {len(unique_emotes)}")

# Create the final shortened list
final_emote_list = unique_emotes

Total unique emotes: 1040


In [9]:
new_emotes = [
    'ome62','Staredown','PentagramOfFarallah','Gloving','footgammaRadiation',
    'RAAAAAAAAGH','praise','doroRage','AndreSmithing','omePIECE','Rime',
    'doroPIECE','kaiReading','Deadge','furi','blub','duh','nuh','gopissgirl',
    'plong','Awesome','o','feaky','DoroThinking','evol','doro18','staycalm',
    'omeFOWL','s','fairs','deal','moshimoshi','gn'
]

final_emote_list = list(set(final_emote_list) | set(new_emotes))


In [10]:
new_emotes = [
    'ae','doroModCheck','Sup','DOSESHEKNOW','BENNED','bitrate','cortisolspike','mlem','catWait','gkeyWave','paulPls','monkeyPls','mountainlion','Walking','KILLTHATBOY','SenChest','peculiar','Senny','wideSpeedPoint','Emotional','sayWALLAHI','doroPls2','sennyTurkey','paulPiece','SCRAPETHATSHITDORO'
]

final_emote_list = list(set(final_emote_list) | set(new_emotes))

In [11]:
# Path to your Excel file
file_path = '../DorozeaStreamStats.xlsx'

# Reading a specific sheet with custom column names and skipping the first two rows
steamStats = pd.read_excel(file_path, sheet_name='Munkalap1', header=None, names=['stream', 'avg', 'max','follow','games'], skiprows=1)

In [12]:
import pandas as pd
from collections import Counter

# --- 1. PREPARATION ---
# Make sure your date column is actually a datetime object so we can calculate durations
data['date'] = pd.to_datetime(data['date'])

# Convert the emote list to a set for much faster lookup speeds
emote_set = set(final_emote_list)

# Define the stream ranges (Python's range is exclusive at the end, so add 1)
la_noire_streams = list(range(156, 167)) # 156 to 166
la_streams = list(range(538, 559))       # 538 to 558

# Pre-calculate the absolute first stream for EVERY user in the entire dataset
# This is needed to figure out true "first-time chatters" for an arc
user_first_stream = data.groupby('user')['stream'].min()

# --- 2. CALCULATION FUNCTION ---
def analyze_arc(arc_name, stream_list, df_chat, df_stats):
    # Filter chat data and stats data for just these streams
    arc_chat = df_chat[df_chat['stream'].isin(stream_list)]
    arc_stats = df_stats[df_stats['stream'].isin(stream_list)]
    
    # 1. Total Duration (sum of last message - first message per stream)
    stream_durations = arc_chat.groupby('stream')['date'].agg(['min', 'max'])
    stream_durations['duration'] = stream_durations['max'] - stream_durations['min']
    total_duration = stream_durations['duration'].sum()
    
    # 2. Unique Chatters
    unique_chatters = arc_chat['user'].nunique()
    
    # 3. First-time Chatters (Users whose VERY FIRST message ever falls in this arc)
    first_time_chatters = user_first_stream[user_first_stream.isin(stream_list)].count()
    
    # 4. Average Message Speed (Messages per minute)
    total_messages = len(arc_chat)
    total_minutes = total_duration.total_seconds() / 60
    # Prevent division by zero just in case
    avg_msg_speed = (total_messages / total_minutes) if total_minutes > 0 else 0
    
    # 5. Total Follows
    total_follows = arc_stats['follow'].sum()
    
    # 6. Top 10 Chatters
    top_10_chatters = arc_chat['user'].value_counts().head(10)
    
    # 7. Top 10 Emotes
    # Split all messages by space, explode into a single column of words, and drop NaNs
    all_words = arc_chat['message'].dropna().astype(str).str.split(expand=False).explode()
    # Filter to only keep words that exist in our emote set
    emotes_used = all_words[all_words.isin(emote_set)]
    top_10_emotes = emotes_used.value_counts().head(10)
    
    # --- PRINT RESULTS ---
    print(f"=== {arc_name.upper()} ARC ===")
    print(f"Total Duration: {total_duration}")
    print(f"Total Messages: {total_messages}")
    print(f"Unique Chatters: {unique_chatters}")
    print(f"First-Time Chatters: {first_time_chatters}")
    print(f"Avg Message Speed: {avg_msg_speed:.2f} messages/min")
    print(f"Total Follows: {total_follows}")
    
    print("\nTop 10 Chatters:")
    print(top_10_chatters.to_string())
    
    print("\nTop 10 Emotes:")
    print(top_10_emotes.to_string())
    print("-" * 40 + "\n")

# --- 3. RUN THE ANALYSIS ---
analyze_arc("LA Noire (156-166)", la_noire_streams, data, steamStats)
analyze_arc("LA Trip (538-558)", la_streams, data, steamStats)

=== LA NOIRE (156-166) ARC ===
Total Duration: 2 days 14:12:11
Total Messages: 61168
Unique Chatters: 3629
First-Time Chatters: 1727
Avg Message Speed: 16.39 messages/min
Total Follows: 1442

Top 10 Chatters:
user
elluiti           2697
JBIN2036          2570
CrazeE420xd       1612
Martin_Gales      1533
trek44_           1484
balintboss        1451
riesenklotz       1305
catsspurr         1098
StreamElements    1044
fuzok4            1009

Top 10 Emotes:
message
ome44     8085
LOL       1444
OOOO      1078
mhm        899
hi         866
WW         769
ome18      581
omE        523
ome5       504
Prayge     496
----------------------------------------

=== LA TRIP (538-558) ARC ===
Total Duration: 2 days 09:28:27
Total Messages: 105985
Unique Chatters: 5744
First-Time Chatters: 1662
Avg Message Speed: 30.73 messages/min
Total Follows: 1073

Top 10 Chatters:
user
FilipStayout    5794
Odah_02         4741
polimpompis     4551
erdeedge        4418
rafa30___       4248
JBIN2036        3531


In [13]:
# --- AUDIENCE RETENTION ---
# 1. Get unique users from both arcs
la_noire_users = set(data[data['stream'].isin(la_noire_streams)]['user'])
la_trip_users = set(data[data['stream'].isin(la_streams)]['user'])

# 2. Find the overlap (users who chatted in BOTH)
retained_users = la_noire_users.intersection(la_trip_users)

# 3. Calculate the retention percentage
if len(la_noire_users) > 0:
    retention_rate = (len(retained_users) / len(la_noire_users)) * 100
    print(f"Audience Retention: {retention_rate:.1f}% of LA Noire chatters returned for the LA Trip.")

# --- THE LURKER RATIO (Avg Viewers vs. Unique Chatters) ---
# Calculate for LA Noire
noire_stats = steamStats[steamStats['stream'].isin(la_noire_streams)]
noire_avg_viewers = noire_stats['avg'].mean()
noire_chatters = len(la_noire_users)

# Calculate for LA Trip
la_stats = steamStats[steamStats['stream'].isin(la_streams)]
la_avg_viewers = la_stats['avg'].mean()
la_chatters = len(la_trip_users)

print(f"\n=== LURKER METRICS ===")
print(f"LA Noire: ~{noire_avg_viewers:.0f} avg viewers | {noire_chatters} unique chatters")
print(f"LA Trip: ~{la_avg_viewers:.0f} avg viewers | {la_chatters} unique chatters")

Audience Retention: 6.0% of LA Noire chatters returned for the LA Trip.

=== LURKER METRICS ===
LA Noire: ~419 avg viewers | 3629 unique chatters
LA Trip: ~947 avg viewers | 5744 unique chatters


In [16]:
def analyze_advanced_metrics(arc_name, stream_list, df_chat, df_stats):
    # Filter data for the specific arc using .copy() to prevent SettingWithCopy warnings
    arc_chat = df_chat[df_chat['stream'].isin(stream_list)].copy()
    arc_stats = df_stats[df_stats['stream'].isin(stream_list)].copy()
    
    print(f"=== {arc_name.upper()} ADVANCED METRICS ===")
    
    # 1. Core Chatter Concentration (Top 10% of chatters)
    user_counts = arc_chat['user'].value_counts()
    total_msgs = len(arc_chat)
    # Find how many users make up the top 10% (minimum of 1)
    top_10_pct_count = max(1, int(len(user_counts) * 0.10))
    top_10_msgs = user_counts.head(top_10_pct_count).sum()
    concentration = (top_10_msgs / total_msgs) * 100 if total_msgs > 0 else 0
    
    print(f"Core Chatter Concentration: The top 10% of chatters sent {concentration:.1f}% of all messages.")

    # 1.5. Core Chatter Concentration (Top 10% of chatters)
    
    # Find how many users make up the top 10% (minimum of 1)
    top_1_pct_count = max(1, int(len(user_counts) * 0.01))
    top_1_msgs = user_counts.head(top_1_pct_count).sum()
    concentration = (top_1_msgs / total_msgs) * 100 if total_msgs > 0 else 0
    
    print(f"Core Chatter Concentration: The top 1% of chatters sent {concentration:.1f}% of all messages.")
    
    # 2. Message Length Analysis
    # Convert to string and calculate length using pandas vectorized .str.len()
    avg_char_length = arc_chat['message'].astype(str).str.len().mean()
    print(f"Average Message Length: {avg_char_length:.1f} characters.")
    
    # 3. Follow-to-Peak Conversion Rate
    # Temporarily replace any 0 max viewers with 1 to avoid division by zero errors
    safe_max = arc_stats['max'].replace(0, 1)
    arc_stats['conversion'] = (arc_stats['follow'] / safe_max) * 100
    avg_conversion = arc_stats['conversion'].mean()
    print(f"Follow-to-Peak Conversion Rate: {avg_conversion:.2f}% avg conversion per stream.")
    
        
    print("-" * 50 + "\n")

# --- RUN THE ADVANCED ANALYSIS ---
# (Make sure la_noire_streams, la_streams, data, and steamStats are loaded from the previous script)
analyze_advanced_metrics("LA Noire (156-166)", la_noire_streams, data, steamStats)
analyze_advanced_metrics("LA Trip (538-558)", la_streams, data, steamStats)

=== LA NOIRE (156-166) ADVANCED METRICS ===
Core Chatter Concentration: The top 10% of chatters sent 82.3% of all messages.
Core Chatter Concentration: The top 1% of chatters sent 51.2% of all messages.
Average Message Length: 22.9 characters.
Follow-to-Peak Conversion Rate: 23.06% avg conversion per stream.
--------------------------------------------------

=== LA TRIP (538-558) ADVANCED METRICS ===
Core Chatter Concentration: The top 10% of chatters sent 86.6% of all messages.
Core Chatter Concentration: The top 1% of chatters sent 63.5% of all messages.
Average Message Length: 18.8 characters.
Follow-to-Peak Conversion Rate: 4.29% avg conversion per stream.
--------------------------------------------------



In [20]:
def analyze_community_health(arc_name, stream_list, df_chat, df_stats):
    # Filter the data for this specific arc
    arc_chat = df_chat[df_chat['stream'].isin(stream_list)]
    arc_stats = df_stats[df_stats['stream'].isin(stream_list)]
    
    print(f"=== {arc_name.upper()} COMMUNITY HEALTH ===")
    
    # --- 1. Chatter Dedication (>1 Message Filter) ---
    total_arc_streams = len(arc_chat['stream'].unique())
    
    # Count total messages per user in this arc
    user_msg_counts = arc_chat.groupby('user').size()
    # Create a list of users who sent more than 1 message
    dedicated_users = user_msg_counts[user_msg_counts > 10].index
    
    # Filter our chat data to ONLY include those dedicated users
    dedicated_chat = arc_chat[arc_chat['user'].isin(dedicated_users)]
    
    # Count how many unique streams each dedicated user appeared in
    streams_per_user = dedicated_chat.groupby('user')['stream'].nunique()
    
    # Prevent errors just in case there's an empty dataset
    avg_attendance = streams_per_user.mean() if not streams_per_user.empty else 0
    
    print(f"Chatter Dedication: The average dedicated chatter (>1 msg) showed up to {avg_attendance:.1f} out of {total_arc_streams} streams.")
    
    # --- 2. Talkativeness Score (Messages per Chatter) ---
    # We still use the un-filtered chat here to get the true overall average
    unique_chatters = arc_chat['user'].nunique()
    total_messages = len(arc_chat)
    msgs_per_chatter = total_messages / unique_chatters if unique_chatters > 0 else 0
    
    print(f"Talkativeness Score: {msgs_per_chatter:.1f} messages sent per active chatter.")
    
    # --- 3. Growth Efficiency (Follows per Hour) ---
    # Calculate total duration in hours
    stream_durations = arc_chat.groupby('stream')['date'].agg(['min', 'max'])
    stream_durations['duration'] = stream_durations['max'] - stream_durations['min']
    total_hours = stream_durations['duration'].sum().total_seconds() / 3600
    
    total_follows = arc_stats['follow'].sum()
    follows_per_hour = total_follows / total_hours if total_hours > 0 else 0
    
    print(f"Growth Efficiency: Gained {follows_per_hour:.1f} follows per hour of streaming.")
    print("-" * 50 + "\n")

# --- RUN THE HEALTH ANALYSIS ---
analyze_community_health("LA Noire (156-166)", la_noire_streams, data, steamStats)
analyze_community_health("LA Trip (538-558)", la_streams, data, steamStats)

=== LA NOIRE (156-166) COMMUNITY HEALTH ===
Chatter Dedication: The average dedicated chatter (>1 msg) showed up to 4.1 out of 11 streams.
Talkativeness Score: 16.9 messages sent per active chatter.
Growth Efficiency: Gained 23.2 follows per hour of streaming.
--------------------------------------------------

=== LA TRIP (538-558) COMMUNITY HEALTH ===
Chatter Dedication: The average dedicated chatter (>1 msg) showed up to 5.1 out of 21 streams.
Talkativeness Score: 18.5 messages sent per active chatter.
Growth Efficiency: Gained 18.7 follows per hour of streaming.
--------------------------------------------------



In [21]:
import numpy as np

def analyze_chat_culture(arc_name, stream_list, df_chat, df_stats, user_first_stream):
    # Filter the data for this specific arc
    arc_chat = df_chat[df_chat['stream'].isin(stream_list)].copy()
    arc_stats = df_stats[df_stats['stream'].isin(stream_list)].copy()
    
    print(f"=== {arc_name.upper()} CHAT CULTURE & VIBE ===")
    
    # Ensure messages are strings to prevent errors
    text_msgs = arc_chat['message'].dropna().astype(str)
    
    # --- 1. Hype Temperature (Caps Lock Ratio) ---
    # Only look at messages 3 characters or longer (ignores "W", "F", "L")
    long_msgs = text_msgs[text_msgs.str.len() >= 3]
    caps_msgs = long_msgs[long_msgs.str.isupper()]
    hype_ratio = (len(caps_msgs) / len(long_msgs)) * 100 if len(long_msgs) > 0 else 0
    print(f"Hype Temperature: {hype_ratio:.1f}% of messages were screaming in ALL CAPS.")
    
    # --- 2. Curiosity / Backseating Index ---
    # Count messages containing a question mark
    question_msgs = text_msgs[text_msgs.str.contains(r'\?', regex=True)]
    question_ratio = (len(question_msgs) / len(text_msgs)) * 100 if len(text_msgs) > 0 else 0
    print(f"Curiosity Index: {question_ratio:.1f}% of messages contained a question mark.")
    
    # --- 3. The "Dead Air" Index ---
    # Sort chronologically, then find the time difference between each consecutive message per stream
    arc_chat = arc_chat.sort_values(by=['stream', 'date'])
    arc_chat['time_diff'] = arc_chat.groupby('stream')['date'].diff().dt.total_seconds()
    
    # Find the longest gap for each stream, then average those out
    max_gaps = arc_chat.groupby('stream')['time_diff'].max()
    avg_max_gap = max_gaps.mean()
    absolute_max_gap = max_gaps.max()
    print(f"Dead Air Index: The average 'longest silence' per stream was {avg_max_gap:.0f} seconds.")
    print(f"  -> (The absolute longest chat silence in this arc was {absolute_max_gap:.0f} seconds.)")
    
    # --- 4. Introvert vs Extrovert Ratio ---
    first_timers = user_first_stream[user_first_stream.isin(stream_list)].count()
    total_follows = arc_stats['follow'].sum()
    
    if total_follows > 0:
        extrovert_ratio = first_timers / total_follows
        print(f"Introvert/Extrovert Ratio: For every 1 follow, there were {extrovert_ratio:.1f} first-time chatters.")
    else:
        print("Introvert/Extrovert Ratio: 0 follows in this dataset, cannot calculate ratio.")
        
    print("-" * 50 + "\n")

# --- RUN THE CULTURE ANALYSIS ---
# (Make sure to pass 'user_first_stream' which we calculated in the very first block of code!)
analyze_chat_culture("LA Noire (156-166)", la_noire_streams, data, steamStats, user_first_stream)
analyze_chat_culture("LA Trip (538-558)", la_streams, data, steamStats, user_first_stream)

=== LA NOIRE (156-166) CHAT CULTURE & VIBE ===
Hype Temperature: 10.9% of messages were screaming in ALL CAPS.
Curiosity Index: 8.8% of messages contained a question mark.
Dead Air Index: The average 'longest silence' per stream was 72 seconds.
  -> (The absolute longest chat silence in this arc was 159 seconds.)
Introvert/Extrovert Ratio: For every 1 follow, there were 1.2 first-time chatters.
--------------------------------------------------

=== LA TRIP (538-558) CHAT CULTURE & VIBE ===
Hype Temperature: 12.0% of messages were screaming in ALL CAPS.
Curiosity Index: 6.4% of messages contained a question mark.
Dead Air Index: The average 'longest silence' per stream was 47 seconds.
  -> (The absolute longest chat silence in this arc was 303 seconds.)
Introvert/Extrovert Ratio: For every 1 follow, there were 1.5 first-time chatters.
--------------------------------------------------



In [22]:
def analyze_financial_roi(arc_name, stream_list, df_chat, df_stats, total_cost_usd):
    # Filter the data for this specific arc
    arc_chat = df_chat[df_chat['stream'].isin(stream_list)]
    arc_stats = df_stats[df_stats['stream'].isin(stream_list)]
    
    print(f"=== {arc_name.upper()} FINANCIAL ROI ===")
    print(f"Total Investment: ${total_cost_usd:,.2f}\n")
    
    # --- 1. Cost per Hour of Content ---
    stream_durations = arc_chat.groupby('stream')['date'].agg(['min', 'max'])
    stream_durations['duration'] = stream_durations['max'] - stream_durations['min']
    total_hours = stream_durations['duration'].sum().total_seconds() / 3600
    
    cost_per_hour = total_cost_usd / total_hours if total_hours > 0 else 0
    print(f"Production Cost: ${cost_per_hour:.2f} per hour of streamed content.")
    
    # --- 2. Cost per Follower (Customer Acquisition Cost - CAC) ---
    total_follows = arc_stats['follow'].sum()
    cac = total_cost_usd / total_follows if total_follows > 0 else 0
    print(f"Cost per Follower (CAC): ${cac:.2f} spent to acquire each new follower.")
    
    # --- 3. Cost per Active Chatter (Engagement Cost) ---
    unique_chatters = arc_chat['user'].nunique()
    cost_per_chatter = total_cost_usd / unique_chatters if unique_chatters > 0 else 0
    print(f"Cost per Active Chatter: ${cost_per_chatter:.2f} spent per engaged viewer.")
    
    # --- 4. Cost per Message ---
    total_messages = len(arc_chat)
    cost_per_message = total_cost_usd / total_messages if total_messages > 0 else 0
    print(f"Cost per Message: ${cost_per_message:.4f} spent for every line of chat.")
    print("-" * 50 + "\n")

# --- RUN THE FINANCIAL ANALYSIS ---
analyze_financial_roi("LA Noire (156-166)", la_noire_streams, data, steamStats, 20.00)
analyze_financial_roi("LA Trip (538-558)", la_streams, data, steamStats, 10000.00)

=== LA NOIRE (156-166) FINANCIAL ROI ===
Total Investment: $20.00

Production Cost: $0.32 per hour of streamed content.
Cost per Follower (CAC): $0.01 spent to acquire each new follower.
Cost per Active Chatter: $0.01 spent per engaged viewer.
Cost per Message: $0.0003 spent for every line of chat.
--------------------------------------------------

=== LA TRIP (538-558) FINANCIAL ROI ===
Total Investment: $10,000.00

Production Cost: $173.99 per hour of streamed content.
Cost per Follower (CAC): $9.32 spent to acquire each new follower.
Cost per Active Chatter: $1.74 spent per engaged viewer.
Cost per Message: $0.0944 spent for every line of chat.
--------------------------------------------------

